In [1]:
import catboost as cb
import pandas as pd 
import sklearn as sk
import numpy as np
import datetime as dt

from dataclasses import dataclass
from sklearn.model_selection import train_test_split
from google.cloud import bigquery

%load_ext watermark
%watermark --packages catboost,pandas,sklearn,numpy,google.cloud.bigquery

catboost 0.26.1
pandas 1.1.5
sklearn 0.23.2
numpy 1.19.5
google.cloud.bigquery unknown


In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("thelook_ecommerce", project="bigquery-public-data")
dataset_ref
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Get a list of available tables 
tables = list(client.list_tables(dataset))

list_of_tables =[table.table_id for table in tables]
print(list_of_tables, dataset_ref)

['distribution_centers', 'events', 'inventory_items', 'order_items', 'orders', 'products', 'users'] DatasetReference('bigquery-public-data', 'thelook_ecommerce')


In [3]:
def BQDb():
    dbName2 = "`bigquery-public-data.thelook_ecommerce`"
    return dbName2
db = BQDb()

query = """
    SELECT
      transactions.user_id,
      products.brand,
      products.category,  -- Fixed typo here
      products.department,
      products.retail_price,
      users.gender,
      users.age,
      users.created_at,
      users.country,
      users.city,
      transactions.created_at  -- Fixed typo here as well
    FROM `bigquery-public-data.thelook_ecommerce.order_items` as transactions
    LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` as users
      ON transactions.user_id = users.id
    LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` as products
      ON transactions.product_id = products.id
    WHERE transactions.status <> 'Cancelled';  -- Make sure to use the correct alias 'transactions'
"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
df = query_job.to_dataframe() 

# Preview results
print(df.head())

   user_id               brand       category department  retail_price gender  \
0    95262  Carol Wright Gifts  Clothing Sets      Women         19.99      F   
1    79942  Carol Wright Gifts  Clothing Sets      Women         19.99      F   
2    84678  Carol Wright Gifts  Clothing Sets      Women         19.99      F   
3    53121  Carol Wright Gifts  Clothing Sets      Women         19.99      F   
4    55000  Carol Wright Gifts  Clothing Sets      Women         19.99      F   

   age                created_at      country  \
0   65 2020-08-31 08:07:00+00:00        China   
1   45 2023-10-21 11:04:00+00:00        China   
2   63 2022-06-18 15:40:00+00:00        China   
3   12 2021-03-06 14:25:00+00:00  South Korea   
4   49 2019-08-13 00:32:00+00:00        China   

                                    city              created_at_1  
0                                Taiyuan 2024-08-07 06:38:11+00:00  
1                                 Suzhou 2024-03-16 09:02:35+00:00  
2          

Given the objective is to predict which brand customers will buy in their next purchase, proceeding as follows:

1. Group purchases chronologically for each customer
2. If a customer has N purchases, we consider the Nth purchase as the target, and the N-1 as our features.
3. We therefore exclude customers with only 1 purchase

In [4]:
recurrent_customers = df.groupby('user_id')['created_at'].count().to_frame("n_purchases")

# Merge with dataset and filter those with more than 1 purchase
df = df.merge(recurrent_customers, left_on='user_id', right_index=True, how='inner')
df = df.query('n_purchases > 1')

# Fill missing values
df.fillna('NA', inplace=True)

target_brands = [
    'Allegra K', 
    'Calvin Klein', 
    'Carhartt', 
    'Hanes', 
    'Volcom', 
    'Nautica', 
    'Quiksilver', 
    'Diesel',
    'Dockers'
]

aggregation_columns = ['brand', 'department', 'category']

# Group purchases by user chronologically
df_agg = (df.sort_values('created_at')
          .groupby(['user_id', 'gender', 'country', 'city', 'age'], as_index=False)[['brand', 'department', 'category']]
          .agg({k: ";".join for k in ['brand', 'department', 'category']})
         )

# Creating the target
df_agg['last_purchase_brand'] = df_agg['brand'].apply(lambda x: x.split(";")[-1])
df_agg['target'] = df_agg['last_purchase_brand'].isin(target_brands)*1

df_agg['age'] = df_agg['age'].astype(float)

# Removing last item of sequence features to avoid target leakage :
for col in aggregation_columns:
    df_agg[col] = df_agg[col].apply(lambda x: ";".join(x.split(";")[:-1]))

In [5]:
df_agg

,user_id,gender,country,city,age,brand,department,category,last_purchase_brand,target
0,4,M,Japan,Yokohama City,53.0,Perry Ellis,Men,Sweaters,Lord Daniels,0
1,8,M,United States,Lexington,36.0,Rip Curl,Men,Swim,Carhartt,1
2,12,F,United States,Philadelphia,69.0,Allegra K;Ice,Women;Women,Blazers & Jackets;Suits,Gypsy Rose,0
3,13,F,United States,Lowell,54.0,Ice;Corset-story,Women;Women,Suits;Intimates,DC,0
4,17,M,China,Jiaozuo,31.0,Diesel,Men,Underwear,Jockey,0
...,...,...,...,...,...,...,...,...,...,...
38572,99992,M,China,Nanjing,26.0,Rodd & Gunn,Men,Sweaters,Retrofit,0
38573,99994,F,France,Cachan,28.0,Hot Chillys;Steve Madden;Lamaze,Women;Women;Women,Active;Outerwear & Coats;Intimates,LabelShopper,0
38574,99995,F,Brasil,Marechal Deodoro,50.0,Jolie,Women,Intimates,Wrangler,0
38575,99997,F,China,Zhangzhou,42.0,Calvin Klein Jeans,Women,Sweaters,Anne Klein,0


In [6]:
#df_agg.drop('last_purchase_category', axis=1, inplace=True)
df_agg.drop('last_purchase_brand', axis=1, inplace=True)
df_agg.drop('user_id', axis = 1, inplace = True)

#spliting data into train and test
df_train, df_val = train_test_split(df_agg, stratify=df_agg['target'], test_size=0.2)
print(f"{len(df_train)} samples in train")

df_val, df_test = train_test_split(df_val, stratify=df_val['target'], test_size=0.5)
print(f"{len(df_val)} samples in val")
print(f"{len(df_test)} samples in test")

30861 samples in train
3858 samples in val
3858 samples in test


In [7]:
X_train, y_train = df_train.iloc[:, :-1], df_train['target']
X_val, y_val = df_val.iloc[:, :-1], df_val['target']
X_test, y_test = df_test.iloc[:, :-1], df_test['target']

In [8]:
# Define features
features = {
    'numerical': ['retail_price', 'age'],
    'static': ['gender', 'country', 'city'],
    'dynamic': ['brand', 'department', 'category']
}

# Building CatBoost "pools", which are datasets
train_pool = cb.Pool(
    X_train,
    y_train,
    cat_features=features.get("static"),
    text_features=features.get("dynamic"),
)

validation_pool = cb.Pool(
    X_val,
    y_val,
    cat_features=features.get("static"),
    text_features=features.get("dynamic"),
)

# Specifying text processing options to handle our text features
text_processing_options = {
    "tokenizers": [
        {"tokenizer_id": "SemiColon", "delimiter": ";", "lowercasing": "false"}
    ],
    "dictionaries": [{"dictionary_id": "Word", "gram_order": "1"}],
    "feature_processing": {
        "default": [
            {
                "dictionaries_names": ["Word"],
                "feature_calcers": ["BoW"],
                "tokenizers_names": ["SemiColon"],
            }
        ],
    },
}

In [9]:
# Model training
model = cb.CatBoostClassifier(
    iterations=2000,
    loss_function="Logloss",
    random_state=45,
    verbose=1,
    auto_class_weights="SqrtBalanced",
    use_best_model=True,
    text_processing=text_processing_options,
    eval_metric='AUC',
)

model.fit(
    train_pool, 
    eval_set=validation_pool, 
    verbose=10
)

Learning rate set to 0.054682
0:	test: 0.5959843	best: 0.5959843 (0)	total: 99.1ms	remaining: 3m 18s
10:	test: 0.5944861	best: 0.5966707 (1)	total: 394ms	remaining: 1m 11s
20:	test: 0.5889545	best: 0.5966707 (1)	total: 653ms	remaining: 1m 1s
30:	test: 0.5924377	best: 0.5966707 (1)	total: 931ms	remaining: 59.1s
40:	test: 0.5879193	best: 0.5966707 (1)	total: 1.21s	remaining: 57.7s
50:	test: 0.5861046	best: 0.5966707 (1)	total: 1.49s	remaining: 57s
60:	test: 0.5920152	best: 0.5966707 (1)	total: 1.77s	remaining: 56.4s
70:	test: 0.5856490	best: 0.5966707 (1)	total: 2.05s	remaining: 55.7s
80:	test: 0.5826326	best: 0.5966707 (1)	total: 2.33s	remaining: 55.1s
90:	test: 0.5803728	best: 0.5966707 (1)	total: 2.6s	remaining: 54.6s
100:	test: 0.5827393	best: 0.5966707 (1)	total: 2.88s	remaining: 54.2s
110:	test: 0.5867557	best: 0.5966707 (1)	total: 3.16s	remaining: 53.9s
120:	test: 0.5888137	best: 0.5966707 (1)	total: 3.44s	remaining: 53.5s
130:	test: 0.5890669	best: 0.5966707 (1)	total: 3.72s	rema

1160:	test: 0.5842285	best: 0.5966707 (1)	total: 34.3s	remaining: 24.8s
1170:	test: 0.5844074	best: 0.5966707 (1)	total: 34.6s	remaining: 24.5s
1180:	test: 0.5850084	best: 0.5966707 (1)	total: 34.9s	remaining: 24.2s
1190:	test: 0.5852368	best: 0.5966707 (1)	total: 35.2s	remaining: 23.9s
1200:	test: 0.5855401	best: 0.5966707 (1)	total: 35.5s	remaining: 23.6s
1210:	test: 0.5852659	best: 0.5966707 (1)	total: 35.8s	remaining: 23.3s
1220:	test: 0.5855141	best: 0.5966707 (1)	total: 36.1s	remaining: 23.1s
1230:	test: 0.5850183	best: 0.5966707 (1)	total: 36.4s	remaining: 22.8s
1240:	test: 0.5848710	best: 0.5966707 (1)	total: 36.7s	remaining: 22.5s
1250:	test: 0.5851991	best: 0.5966707 (1)	total: 37s	remaining: 22.2s
1260:	test: 0.5850870	best: 0.5966707 (1)	total: 37.3s	remaining: 21.9s
1270:	test: 0.5847769	best: 0.5966707 (1)	total: 37.6s	remaining: 21.6s
1280:	test: 0.5854572	best: 0.5966707 (1)	total: 38s	remaining: 21.3s
1290:	test: 0.5853451	best: 0.5966707 (1)	total: 38.3s	remaining: 21

In [10]:
from sklearn.metrics import roc_auc_score

print(f"ROC-AUC for train set      : {roc_auc_score(y_true=y_train, y_score=model.predict(X_train)):.2f}")
print(f"ROC-AUC for validation set : {roc_auc_score(y_true=y_val, y_score=model.predict(X_val)):.2f}")
print(f"ROC-AUC for test set       : {roc_auc_score(y_true=y_test, y_score=model.predict(X_test)):.2f}")

ROC-AUC for train set      : 0.50
ROC-AUC for validation set : 0.50
ROC-AUC for test set       : 0.50
